# Same process undergone as DataWrangling 7 but retaining columns that were dropped in DataWrangling8 column for datasets in unemployment rate, poverty rate, and education_vehicle

# Importing libraries

In [55]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib

from IPython.core.display import display

# pd.set_option('display.max_columns', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

# Load data and constants

In [56]:
MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD, RI, DC  = [25, 9, 23, 36, 33, 34, 42, 50, 10, 24, 44, 11]
NORTH_EAST = [MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD, RI, DC]
RI, FL,GA, NC, SC = [44, 12, 13, 37, 45] # midatlantic
AL, KY, MS, TN, WV = [1, 21, 29, 47, 45] # south east
IL, IN, IA, KS, MI,MN = [17, 18, 19, 20, 26, 27] # mid_west


daily_aqi_by_county_2017 = pd.read_csv('../../data/raw/daily_aqi_by_county_2017.csv')
daily_aqi_by_county_2018 = pd.read_csv('../../data/raw/daily_aqi_by_county_2018.csv')
daily_aqi_by_county_2019 = pd.read_csv('../../data/raw/daily_aqi_by_county_2019.csv')

county_demographic = pd.read_csv('../../data/raw/county_cc-est2019-alldata.csv', encoding='latin-1')

unemployment = pd.read_excel('../../data/raw/Unemployment.xls', header=None, sheet_name=0)

# Small area income and poverty estimates
saipe2017 = pd.read_excel('../../data/raw/saipe_economic/est17all.xls')
saipe2018 = pd.read_excel('../../data/raw/saipe_economic/est18all.xls')
saipe2019 = pd.read_excel('../../data/raw/saipe_economic/est19all.xls')

educ_vehicle_2017 = pd.read_csv('../../data/processed/acs1_edu_vehicle2017.csv')
educ_vehicle_2018 = pd.read_csv('../../data/processed/acs1_edu_vehicle2018.csv')
educ_vehicle_2019 = pd.read_csv('../../data/processed/acs1_edu_vehicle2019.csv')

annual_income_by_county = pd.read_csv('../../data/raw/CAINC1__ALL_AREAS_1969_2019.csv', encoding='latin-1')


In [57]:
def remove_col(df, col_name):
    '''Returns a dataframe with removed column or columns from old dataframe'''
    new_df = df.copy()

    if(type(col_name) != str and len(col_name) > 1):
        for index in col_name:
            new_df = new_df.drop(str(index), axis=1)
    else:
        new_df = new_df.drop(str(col_name), axis=1)
    return new_df

def get_NE_States(df, col):
    '''Method used to return North Eastern US states'''
    NE_States = (df[col] == MA) | \
                (df[col] == CT) | \
                (df[col] == ME) | \
                (df[col] == NY) | \
                (df[col] == NH) | \
                (df[col] == NJ) | \
                (df[col] == PA) | \
                (df[col] == VT) | \
                (df[col] == DE) | \
                (df[col] == MD) | \
                (df[col] == RI) | \
                (df[col] == DC)
    return NE_States
def get_df_with_geofips(df, state_code, county_code):
    ''' Returns dataframe with geofips column '''
    state_code = df[state_code].astype(str).values
    county_code = df[county_code].astype(str).values

    geofips = []
    for stateID, countyID in zip(state_code, county_code):
        id = ''
        stateLen = len(stateID)
        countyLen = len(countyID)
        if(stateLen < 2):
            id += '0' + stateID
        else:
            id += stateID
        if(countyLen == 1):
            id += '00' + countyID
        elif(countyLen == 2):
            id += '0' + countyID
        else:
            id += countyID
        geofips.append(id)

    df.insert(loc=0, column='GeoFIPS', value=geofips)
    return df

# Clean up Unemployment Data


In [58]:
display(unemployment)
unemployment2017_2019 = unemployment[4:]
unemployment2017_2019.reset_index(drop=True, inplace=True)
unemployment_header = unemployment2017_2019.iloc[0, :].values
unemployment2017_2019.columns = unemployment_header
unemployment2017_2019 = unemployment2017_2019.drop([0])
display(unemployment2017_2019)

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
0,Unemployment and median household income for t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sources: Unemployment: U.S. Department of Labo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,13117,11750,1367,10.4,13037,11791,1246,9.6,NaN,NaN
3276,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2718,2269,449,16.5,2585,2406,179,6.9,NaN,NaN
3277,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7311,6154,1157,15.8,7406,6231,1175,15.9,NaN,NaN
3278,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,8762,7509,1253,14.3,8691,7552,1139,13.1,NaN,NaN


,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
1,00000,US,United States,NaN,NaN,NaN,142601667,136904680,5696987,3.99504,...,161389026,155102319,6286707,3.89537,163100055,157115247,5984808,3.66941,65712,NaN
2,01000,AL,Alabama,NaN,NaN,NaN,2133223,2035594,97629,4.6,...,2216627,2130845,85782,3.9,2241747,2174483,67264,3,51771,100
3,01001,AL,"Autauga County, AL",2,2,1,21720,20846,874,4,...,26196,25261,935,3.6,26172,25458,714,2.7,58233,112.482
4,01003,AL,"Baldwin County, AL",3,2,1,69533,66971,2562,3.7,...,95233,91809,3424,3.6,97328,94675,2653,2.7,59871,115.646
5,01005,AL,"Barbour County, AL",6,6,0,11373,10748,625,5.5,...,8414,7987,427,5.1,8537,8213,324,3.8,35972,69.4829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,13117,11750,1367,10.4,13037,11791,1246,9.6,NaN,NaN
3272,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2718,2269,449,16.5,2585,2406,179,6.9,NaN,NaN
3273,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7311,6154,1157,15.8,7406,6231,1175,15.9,NaN,NaN
3274,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,8762,7509,1253,14.3,8691,7552,1139,13.1,NaN,NaN


* fix header columns of csv file
* remove empty rows

In [59]:
# used to remove other year data to keep only 2017-2019
display(unemployment2017_2019.columns)
cols_to_remove = unemployment2017_2019.iloc[:, 3:74].columns
unemployment2017_2019 = remove_col(unemployment2017_2019, cols_to_remove)


# unemployment2017_2019

Index(['fips_txt', 'Stabr', 'area_name', 'Rural_urban_continuum_code_2013',
       'Urban_influence_code_2013', 'Metro_2013', 'Civilian_labor_force_2000',
       'Employed_2000', 'Unemployed_2000', 'Unemployment_rate_2000',
       'Civilian_labor_force_2001', 'Employed_2001', 'Unemployed_2001',
       'Unemployment_rate_2001', 'Civilian_labor_force_2002', 'Employed_2002',
       'Unemployed_2002', 'Unemployment_rate_2002',
       'Civilian_labor_force_2003', 'Employed_2003', 'Unemployed_2003',
       'Unemployment_rate_2003', 'Civilian_labor_force_2004', 'Employed_2004',
       'Unemployed_2004', 'Unemployment_rate_2004',
       'Civilian_labor_force_2005', 'Employed_2005', 'Unemployed_2005',
       'Unemployment_rate_2005', 'Civilian_labor_force_2006', 'Employed_2006',
       'Unemployed_2006', 'Unemployment_rate_2006',
       'Civilian_labor_force_2007', 'Employed_2007', 'Unemployed_2007',
       'Unemployment_rate_2007', 'Civilian_labor_force_2008', 'Employed_2008',
       'Unemploy

In [60]:
unemployment_rates_2017_2019 = unemployment2017_2019.iloc[:,:-2]
display(unemployment_rates_2017_2019)

,fips_txt,Stabr,area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019
1,00000,US,United States,160158564,153183461,6975103,4.35512,161389026,155102319,6286707,3.89537,163100055,157115247,5984808,3.66941
2,01000,AL,Alabama,2186869,2090373,96496,4.4,2216627,2130845,85782,3.9,2241747,2174483,67264,3
3,01001,AL,"Autauga County, AL",26075,25062,1013,3.9,26196,25261,935,3.6,26172,25458,714,2.7
4,01003,AL,"Baldwin County, AL",92456,88711,3745,4.1,95233,91809,3424,3.6,97328,94675,2653,2.7
5,01005,AL,"Barbour County, AL",8349,7863,486,5.8,8414,7987,427,5.1,8537,8213,324,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,72145,PR,"Vega Baja Municipio, PR",13414,11746,1668,12.4,13117,11750,1367,10.4,13037,11791,1246,9.6
3272,72147,PR,"Vieques Municipio, PR",3027,2589,438,14.5,2718,2269,449,16.5,2585,2406,179,6.9
3273,72149,PR,"Villalba Municipio, PR",7557,6090,1467,19.4,7311,6154,1157,15.8,7406,6231,1175,15.9
3274,72151,PR,"Yabucoa Municipio, PR",9000,7512,1488,16.5,8762,7509,1253,14.3,8691,7552,1139,13.1


* remove all other column data for years 2000 - 2016
* make a df for only unemployment rates

In [61]:
# Retain only counties and not regions / individual state

UE_counties = unemployment_rates_2017_2019['area_name'].str.contains(',')
UE_county_rate = unemployment_rates_2017_2019.loc[UE_counties]

In [62]:
UE_county_rate.dropna(inplace=True)
UE_county_rate.reset_index(drop=True, inplace=True)

UE_county_rate.rename(columns={'fips_txt':'GeoFIPS'}, inplace=True)

<ipython-input-62-53a924a2cf8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate.dropna(inplace=True)
/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


* 3 alaskan counties show unknown values;
* These values will not be mergeable with aqi data as it we do not aqi data on those areas
* drop these rows
* Rename fips_txt to GeoFIPS


* we have 0 duplicates

In [63]:
display(UE_county_rate.info())

UE_county_rate['Unemployment_rate_2017'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2017'], downcast='float')
UE_county_rate['Unemployment_rate_2018'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2018'], downcast='float')
UE_county_rate['Unemployment_rate_2019'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2019'], downcast='float')


for col in UE_county_rate.columns[[3,4,5,7,8,9,11,12,13]]:
    UE_county_rate[col] = pd.to_numeric(UE_county_rate[col], downcast='integer')


UE_county_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   GeoFIPS                    3218 non-null   object
 1   Stabr                      3218 non-null   object
 2   area_name                  3218 non-null   object
 3   Civilian_labor_force_2017  3218 non-null   object
 4   Employed_2017              3218 non-null   object
 5   Unemployed_2017            3218 non-null   object
 6   Unemployment_rate_2017     3218 non-null   object
 7   Civilian_labor_force_2018  3218 non-null   object
 8   Employed_2018              3218 non-null   object
 9   Unemployed_2018            3218 non-null   object
 10  Unemployment_rate_2018     3218 non-null   object
 11  Civilian_labor_force_2019  3218 non-null   object
 12  Employed_2019              3218 non-null   object
 13  Unemployed_2019            3218 non-null   object
 14  Unemploy

None

<ipython-input-63-1d790d0a8427>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate['Unemployment_rate_2017'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2017'], downcast='float')
<ipython-input-63-1d790d0a8427>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate['Unemployment_rate_2018'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2018'], downcast='float')
<ipython-input-63-1d790d0a8427>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

* convert Unemployment rate columsn to be type float


* In 2017, it appears that Unemployment rates from 2.5 to 7.5 are normal
* data is skewed to right

### Retrieve NE Unemployment Data


In [64]:
NE_States = (UE_county_rate['Stabr'] == 'MA') | \
            (UE_county_rate['Stabr'] == 'CT') | \
            (UE_county_rate['Stabr'] == 'ME') | \
            (UE_county_rate['Stabr'] == 'NY') | \
            (UE_county_rate['Stabr'] == 'NH') | \
            (UE_county_rate['Stabr'] == 'NJ') | \
            (UE_county_rate['Stabr'] == 'PA') | \
            (UE_county_rate['Stabr'] == 'VT') | \
            (UE_county_rate['Stabr'] == 'DE') | \
            (UE_county_rate['Stabr'] == 'MD') | \
            (UE_county_rate['Stabr'] == 'RI') | \
            (UE_county_rate['Stabr'] == 'DC')

NE_unemploy_rate_2017_2019 = UE_county_rate.loc[NE_States]

display(NE_unemploy_rate_2017_2019)

,GeoFIPS,Stabr,area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019
308,09001,CT,"Fairfield County, CT",479458,457625,21833,4.6,479179,460045,19134,4.0,481023,463547,17476,3.6
309,09003,CT,"Hartford County, CT",477390,454205,23185,4.9,478660,458138,20522,4.3,483303,464533,18770,3.9
310,09005,CT,"Litchfield County, CT",104051,99522,4529,4.4,104604,100568,4036,3.9,105182,101543,3639,3.5
311,09007,CT,"Middlesex County, CT",92583,88817,3766,4.1,92811,89517,3294,3.5,93696,90703,2993,3.2
312,09009,CT,"New Haven County, CT",457800,434645,23155,5.1,458799,438485,20314,4.4,461613,443514,18099,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813,50019,VT,"Orleans County, VT",13601,12920,681,5.0,13501,12924,577,4.3,13275,12713,562,4.2
2814,50021,VT,"Rutland County, VT",31096,30030,1066,3.4,30752,29828,924,3.0,30446,29608,838,2.8
2815,50023,VT,"Washington County, VT",34260,33273,987,2.9,34512,33658,854,2.5,34360,33586,774,2.3
2816,50025,VT,"Windham County, VT",22823,22132,691,3.0,22470,21856,614,2.7,21977,21416,561,2.6


* Northeastern counties of Unemployment data is created

# clean up saipe data 2017


In [65]:
saipe_header = saipe2017.iloc[2, :].values
saipe_header
saipe2017.columns = saipe_header
saipe2017.drop([0,1,2], inplace=True)
saipe2017.reset_index(drop=True, inplace=True)
display(saipe2017)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,42583651,42342619,42824683,13.4,13.3,13.5,...,17.5,60336,60250,60422,3932969,3880645,3985293,20.2,19.9,20.5
1,01,000,AL,Alabama,802263,784517,820009,16.9,16.5,17.3,...,23.8,48193,47451,48935,78986,75009,82963,27.7,26.3,29.1
2,01,001,AL,Autauga County,7390,6147,8633,13.4,11.1,15.7,...,22.1,58343,52121,64565,.,.,.,.,.,.
3,01,003,AL,Baldwin County,21199,17444,24954,10.1,8.3,11.9,...,17.9,56607,52439,60775,.,.,.,.,.,.
4,01,005,AL,Barbour County,7414,6325,8503,33.4,28.5,38.3,...,55,32490,29218,35762,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,4253,3540,4966,10,8.3,11.7,...,14,75590,69190,81990,.,.,.,.,.,.
3190,56,039,WY,Teton County,1348,1021,1675,5.8,4.4,7.2,...,7.9,90145,80434,99856,.,.,.,.,.,.
3191,56,041,WY,Uinta County,2067,1623,2511,10.2,8,12.4,...,13.4,67404,60925,73883,.,.,.,.,.,.
3192,56,043,WY,Washakie County,845,632,1058,10.7,8,13.4,...,17.3,57989,53142,62836,.,.,.,.,.,.


In [66]:
saipe2017.columns
saipe2017.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2017['Name'].str.contains('County')
pov_rate_2017 = saipe2017.loc[saipe_county].iloc[:, :-6]
pov_rate_2017.reset_index(drop=True, inplace=True)
display(pov_rate_2017)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,"Poverty Estimate, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound
0,01,001,AL,Autauga County,7390,6147,8633,13.4,11.1,15.7,...,22.8,1842,1492,2192,18.6,15.1,22.1,58343,52121,64565
1,01,003,AL,Baldwin County,21199,17444,24954,10.1,8.3,11.9,...,18.3,4871,3641,6101,14.3,10.7,17.9,56607,52439,60775
2,01,005,AL,Barbour County,7414,6325,8503,33.4,28.5,38.3,...,56.9,1904,1660,2148,48.8,42.6,55,32490,29218,35762
3,01,007,AL,Bibb County,4137,3187,5087,20.2,15.5,24.9,...,34,870,641,1099,26.8,19.8,33.8,45795,40924,50666
4,01,009,AL,Blount County,7343,5805,8881,12.8,10.1,15.5,...,23,1763,1307,2219,17.7,13.1,22.3,48253,43784,52722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,4253,3540,4966,10,8.3,11.7,...,15.9,972,769,1175,11.6,9.2,14,75590,69190,81990
3003,56,039,WY,Teton County,1348,1021,1675,5.8,4.4,7.2,...,8.9,189,128,250,6,4.1,7.9,90145,80434,99856
3004,56,041,WY,Uinta County,2067,1623,2511,10.2,8,12.4,...,15.1,445,304,586,10.2,7,13.4,67404,60925,73883
3005,56,043,WY,Washakie County,845,632,1058,10.7,8,13.4,...,18.2,185,125,245,13.1,8.9,17.3,57989,53142,62836


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [67]:
print(pov_rate_2017.isnull().sum())
print(pov_rate_2017.duplicated().sum())
display(pov_rate_2017.info())

State FIPS Code                           0
County FIPS Code                          0
Postal Code                               0
Name                                      0
Poverty Estimate, All Ages                0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, All Ages                 0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Estimate, Age 0-17                0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, Age 0-17                 0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Estimate, Age 5-17 in Families    0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, Age 5-17 in Families     0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Median Household Income         

None

* No duplicates or missing values for poverty rates in 2017
* also convert poverty percent to float

In [68]:
pov_rate_2017.info()

pov_rate_2017.drop(518, inplace=True)

# pov_rate_2017['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2017['Poverty Percent, All Ages'], downcast='float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   State FIPS Code                         3007 non-null   object
 1   County FIPS Code                        3007 non-null   object
 2   Postal Code                             3007 non-null   object
 3   Name                                    3007 non-null   object
 4   Poverty Estimate, All Ages              3007 non-null   object
 5   90% CI Lower Bound                      3007 non-null   object
 6   90% CI Upper Bound                      3007 non-null   object
 7   Poverty Percent, All Ages               3007 non-null   object
 8   90% CI Lower Bound                      3007 non-null   object
 9   90% CI Upper Bound                      3007 non-null   object
 10  Poverty Estimate, Age 0-17              3007 non-null   object
 11  90% 

In [69]:

pov_rate_2017.columns = ['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI LB All Ages',
       '90% CI UB All Ages', 'Poverty Percent, All Ages', '90% CI LB percent',
       '90% CI UB percent', 'Poverty Estimate, Age 0-17',
       '90% CI LB percent 0-17', '90% CI UB 0-17', 'Poverty Percent, Age 0-17',
       '90% CI LB percent 0-17 percent', '90% CI UB 0-17 percent',
       'Poverty Estimate, Age 5-17 in Families', '90% CI LB 5-17 fam',
       '90% CI UB 5-17 fam', 'Poverty Percent, Age 5-17 in Families',
       '90% CI LB 5-17 percent', '90% CI UB percent 5-17 percent', 'Median Household Income',
       '90% CI Lower Bound LB medh inc', '90% CI UB medh inc']

pov_cols = pov_rate_2017.columns[4:]

In [70]:
for col in pov_cols:
    pov_rate_2017[col] = pd.to_numeric(pov_rate_2017[col], downcast='float')

pov_rate_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3006 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   State FIPS Code                         3006 non-null   object 
 1   County FIPS Code                        3006 non-null   object 
 2   Postal Code                             3006 non-null   object 
 3   Name                                    3006 non-null   object 
 4   Poverty Estimate, All Ages              3006 non-null   float32
 5   90% CI LB All Ages                      3006 non-null   float32
 6   90% CI UB All Ages                      3006 non-null   float32
 7   Poverty Percent, All Ages               3006 non-null   float32
 8   90% CI LB percent                       3006 non-null   float32
 9   90% CI UB percent                       3006 non-null   float32
 10  Poverty Estimate, Age 0-17              3006 non-null   floa

* from further inspection it was noted that Kalawao county is now excluded in their survey results starting 2017
* Kalawao county records removed
* Poverty rates in Hawaii appear to around 10-20s

# Repeat clean up steps on saipe data 2018

In [71]:
saipe_header = saipe2018.iloc[2, :].values
saipe_header
saipe2018.columns = saipe_header
saipe2018.drop([0,1,2], inplace=True)
saipe2018.reset_index(drop=True, inplace=True)
display(saipe2018)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,41852315,41619366,42085264,13.1,13,13.2,...,17.2,61937,61843,62031,3758704,3714862,3802546,19.5,19.3,19.7
1,01,000,AL,Alabama,801758,785668,817848,16.8,16.5,17.1,...,23.7,49881,49123,50639,73915,69990,77840,26,24.6,27.4
2,01,001,AL,Autauga County,7587,6334,8840,13.8,11.5,16.1,...,23.9,59338,53628,65048,.,.,.,.,.,.
3,01,003,AL,Baldwin County,21069,17390,24748,9.8,8.1,11.5,...,16.9,57588,54437,60739,.,.,.,.,.,.
4,01,005,AL,Barbour County,6788,5662,7914,30.9,25.8,36,...,45.9,34382,31157,37607,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,3540,2804,4276,8.4,6.7,10.1,...,11.2,73315,67141,79489,.,.,.,.,.,.
3190,56,039,WY,Teton County,1443,1137,1749,6.3,5,7.6,...,7.7,99087,88142,110032,.,.,.,.,.,.
3191,56,041,WY,Uinta County,2010,1585,2435,10,7.9,12.1,...,14,63401,56812,69990,.,.,.,.,.,.
3192,56,043,WY,Washakie County,918,700,1136,11.9,9.1,14.7,...,19.3,55190,49263,61117,.,.,.,.,.,.


In [72]:
saipe2018.columns
saipe2018.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2018['Name'].str.contains('County')
pov_rate_2018 = saipe2018.loc[saipe_county].iloc[:, :-6]
pov_rate_2018.reset_index(drop=True, inplace=True)
display(pov_rate_2018)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,"Poverty Estimate, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound
0,01,001,AL,Autauga County,7587,6334,8840,13.8,11.5,16.1,...,23.5,1891,1469,2313,19.5,15.1,23.9,59338,53628,65048
1,01,003,AL,Baldwin County,21069,17390,24748,9.8,8.1,11.5,...,17.6,4534,3223,5845,13.1,9.3,16.9,57588,54437,60739
2,01,005,AL,Barbour County,6788,5662,7914,30.9,25.8,36,...,52.8,1411,1056,1766,36.7,27.5,45.9,34382,31157,37607
3,01,007,AL,Bibb County,4400,3445,5355,21.8,17.1,26.5,...,34.9,840,606,1074,26.3,19,33.6,46064,41283,50845
4,01,009,AL,Blount County,7527,6045,9009,13.2,10.6,15.8,...,22.5,1520,1053,1987,15.5,10.7,20.3,50412,46157,54667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,3540,2804,4276,8.4,6.7,10.1,...,12.7,697,479,915,8.5,5.8,11.2,73315,67141,79489
3003,56,039,WY,Teton County,1443,1137,1749,6.3,5,7.6,...,8.5,180,126,234,5.9,4.1,7.7,99087,88142,110032
3004,56,041,WY,Uinta County,2010,1585,2435,10,7.9,12.1,...,15.6,454,306,602,10.6,7.2,14,63401,56812,69990
3005,56,043,WY,Washakie County,918,700,1136,11.9,9.1,14.7,...,20.8,191,127,255,14.5,9.7,19.3,55190,49263,61117


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [73]:
print(pov_rate_2017.isnull().sum())
print(pov_rate_2017.duplicated().sum())
display(pov_rate_2017.info())

State FIPS Code                           0
County FIPS Code                          0
Postal Code                               0
Name                                      0
Poverty Estimate, All Ages                0
90% CI LB All Ages                        0
90% CI UB All Ages                        0
Poverty Percent, All Ages                 0
90% CI LB percent                         0
90% CI UB percent                         0
Poverty Estimate, Age 0-17                0
90% CI LB percent 0-17                    0
90% CI UB 0-17                            0
Poverty Percent, Age 0-17                 0
90% CI LB percent 0-17 percent            0
90% CI UB 0-17 percent                    0
Poverty Estimate, Age 5-17 in Families    0
90% CI LB 5-17 fam                        0
90% CI UB 5-17 fam                        0
Poverty Percent, Age 5-17 in Families     0
90% CI LB 5-17 percent                    0
90% CI UB percent 5-17 percent            0
Median Household Income         

None

* No duplicates or missing values for poverty rates in 2017
* also convert poverty percent to float

In [74]:
pov_rate_2018.info()

pov_rate_2018.drop(518, inplace=True)
# pov_rate_2017['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2017['Poverty Percent, All Ages'], downcast='float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   State FIPS Code                         3007 non-null   object
 1   County FIPS Code                        3007 non-null   object
 2   Postal Code                             3007 non-null   object
 3   Name                                    3007 non-null   object
 4   Poverty Estimate, All Ages              3007 non-null   object
 5   90% CI Lower Bound                      3007 non-null   object
 6   90% CI Upper Bound                      3007 non-null   object
 7   Poverty Percent, All Ages               3007 non-null   object
 8   90% CI Lower Bound                      3007 non-null   object
 9   90% CI Upper Bound                      3007 non-null   object
 10  Poverty Estimate, Age 0-17              3007 non-null   object
 11  90% 

In [75]:

pov_rate_2018.columns = ['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI LB All Ages',
       '90% CI UB All Ages', 'Poverty Percent, All Ages', '90% CI LB percent',
       '90% CI UB percent', 'Poverty Estimate, Age 0-17',
       '90% CI LB percent 0-17', '90% CI UB 0-17', 'Poverty Percent, Age 0-17',
       '90% CI LB percent 0-17 percent', '90% CI UB 0-17 percent',
       'Poverty Estimate, Age 5-17 in Families', '90% CI LB 5-17 fam',
       '90% CI UB 5-17 fam', 'Poverty Percent, Age 5-17 in Families',
       '90% CI LB 5-17 percent', '90% CI UB percent 5-17 percent', 'Median Household Income',
       '90% CI Lower Bound LB medh inc', '90% CI UB medh inc']

pov_cols = pov_rate_2018.columns[4:]

In [76]:
for col in pov_cols:
    pov_rate_2018[col] = pd.to_numeric(pov_rate_2018[col], downcast='float')

pov_rate_2018.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3006 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   State FIPS Code                         3006 non-null   object 
 1   County FIPS Code                        3006 non-null   object 
 2   Postal Code                             3006 non-null   object 
 3   Name                                    3006 non-null   object 
 4   Poverty Estimate, All Ages              3006 non-null   float32
 5   90% CI LB All Ages                      3006 non-null   float32
 6   90% CI UB All Ages                      3006 non-null   float32
 7   Poverty Percent, All Ages               3006 non-null   float32
 8   90% CI LB percent                       3006 non-null   float32
 9   90% CI UB percent                       3006 non-null   float32
 10  Poverty Estimate, Age 0-17              3006 non-null   floa

* from further inspection it was noted that Kalawao county is now excluded in their survey results starting 2017
* Kalawao county records removed
* Poverty rates in Hawaii appear to around 10-20s


# Repeat clean up steps for saipe data 2019

In [77]:
saipe_header = saipe2019.iloc[2, :].values
saipe_header
saipe2019.columns = saipe_header
saipe2019.drop([0,1,2], inplace=True)
saipe2019.reset_index(drop=True, inplace=True)
display(saipe2019)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,39490096,39248096,39732096,12.3,12.2,12.4,...,16,65712,65594,65830,3457689,3405854,3509524,18.2,17.9,18.5
1,01,000,AL,Alabama,747478,730491,764465,15.6,15.2,16,...,21.6,51771,51179,52363,69236,65296,73176,24.2,22.8,25.6
2,01,001,AL,Autauga County,6723,5517,7929,12.1,9.9,14.3,...,19.4,58233,52517,63949,.,.,.,.,.,.
3,01,003,AL,Baldwin County,22360,18541,26179,10.1,8.4,11.8,...,17.2,59871,54593,65149,.,.,.,.,.,.
4,01,005,AL,Barbour County,5909,4787,7031,27.1,22,32.2,...,49,35972,31822,40122,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,3453,2743,4163,8.3,6.6,10,...,11.1,80639,73437,87841,.,.,.,.,.,.
3190,56,039,WY,Teton County,1396,1073,1719,6,4.6,7.4,...,6.7,98837,86531,111143,.,.,.,.,.,.
3191,56,041,WY,Uinta County,1699,1264,2134,8.5,6.3,10.7,...,11.1,70756,63191,78321,.,.,.,.,.,.
3192,56,043,WY,Washakie County,845,626,1064,11.1,8.2,14,...,17.4,55122,50050,60194,.,.,.,.,.,.


In [78]:
saipe2019.columns
saipe2019.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2019['Name'].str.contains('County')
pov_rate_2019 = saipe2019.loc[saipe_county].iloc[:, :-6]
pov_rate_2019.reset_index(drop=True, inplace=True)
display(pov_rate_2019)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,"Poverty Estimate, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 5-17 in Families",90% CI Lower Bound,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound
0,01,001,AL,Autauga County,6723,5517,7929,12.1,9.9,14.3,...,20.3,1376,902,1850,14.4,9.4,19.4,58233,52517,63949
1,01,003,AL,Baldwin County,22360,18541,26179,10.1,8.4,11.8,...,17.4,4641,3295,5987,13.3,9.4,17.2,59871,54593,65149
2,01,005,AL,Barbour County,5909,4787,7031,27.1,22,32.2,...,50.8,1468,1114,1822,39.5,30,49,35972,31822,40122
3,01,007,AL,Bibb County,4101,3225,4977,20.3,16,24.6,...,33.1,808,566,1050,25.2,17.7,32.7,47918,42291,53545
4,01,009,AL,Blount County,9324,8037,10611,16.3,14.1,18.5,...,25.3,2067,1641,2493,21.4,17,25.8,52902,46777,59027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,3453,2743,4163,8.3,6.6,10,...,13,644,416,872,8.2,5.3,11.1,80639,73437,87841
3003,56,039,WY,Teton County,1396,1073,1719,6,4.6,7.4,...,7.8,142,83,201,4.7,2.7,6.7,98837,86531,111143
3004,56,041,WY,Uinta County,1699,1264,2134,8.5,6.3,10.7,...,13.1,336,193,479,7.8,4.5,11.1,70756,63191,78321
3005,56,043,WY,Washakie County,845,626,1064,11.1,8.2,14,...,18.9,160,94,226,12.3,7.2,17.4,55122,50050,60194


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [79]:
print(pov_rate_2019.isnull().sum())
print(pov_rate_2019.duplicated().sum())
display(pov_rate_2019.info())

State FIPS Code                           0
County FIPS Code                          0
Postal Code                               0
Name                                      0
Poverty Estimate, All Ages                0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, All Ages                 0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Estimate, Age 0-17                0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, Age 0-17                 0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Estimate, Age 5-17 in Families    0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Poverty Percent, Age 5-17 in Families     0
90% CI Lower Bound                        0
90% CI Upper Bound                        0
Median Household Income         

None

* No duplicates or missing values for poverty rates in 2017
* also convert poverty percent to float

In [80]:
pov_rate_2019.info()

pov_rate_2019.drop(518, inplace=True)
# pov_rate_2017['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2017['Poverty Percent, All Ages'], downcast='float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   State FIPS Code                         3007 non-null   object
 1   County FIPS Code                        3007 non-null   object
 2   Postal Code                             3007 non-null   object
 3   Name                                    3007 non-null   object
 4   Poverty Estimate, All Ages              3007 non-null   object
 5   90% CI Lower Bound                      3007 non-null   object
 6   90% CI Upper Bound                      3007 non-null   object
 7   Poverty Percent, All Ages               3007 non-null   object
 8   90% CI Lower Bound                      3007 non-null   object
 9   90% CI Upper Bound                      3007 non-null   object
 10  Poverty Estimate, Age 0-17              3007 non-null   object
 11  90% 

In [81]:

pov_rate_2019.columns = ['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI LB All Ages',
       '90% CI UB All Ages', 'Poverty Percent, All Ages', '90% CI LB percent',
       '90% CI UB percent', 'Poverty Estimate, Age 0-17',
       '90% CI LB percent 0-17', '90% CI UB 0-17', 'Poverty Percent, Age 0-17',
       '90% CI LB percent 0-17 percent', '90% CI UB 0-17 percent',
       'Poverty Estimate, Age 5-17 in Families', '90% CI LB 5-17 fam',
       '90% CI UB 5-17 fam', 'Poverty Percent, Age 5-17 in Families',
       '90% CI LB 5-17 percent', '90% CI UB percent 5-17 percent', 'Median Household Income',
       '90% CI Lower Bound LB medh inc', '90% CI UB medh inc']

pov_cols = pov_rate_2019.columns[4:]

In [82]:
for col in pov_cols:
    pov_rate_2019[col] = pd.to_numeric(pov_rate_2019[col], downcast='float')

pov_rate_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3006 entries, 0 to 3006
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   State FIPS Code                         3006 non-null   object 
 1   County FIPS Code                        3006 non-null   object 
 2   Postal Code                             3006 non-null   object 
 3   Name                                    3006 non-null   object 
 4   Poverty Estimate, All Ages              3006 non-null   float32
 5   90% CI LB All Ages                      3006 non-null   float32
 6   90% CI UB All Ages                      3006 non-null   float32
 7   Poverty Percent, All Ages               3006 non-null   float32
 8   90% CI LB percent                       3006 non-null   float32
 9   90% CI UB percent                       3006 non-null   float32
 10  Poverty Estimate, Age 0-17              3006 non-null   floa

# Clean education and vehicle 2017 dataset

In [83]:
educ_vehicle_2017.shape
educ_vehicle_2017.columns

educ_vehicle_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        837 non-null    int64  
 1   NAME              837 non-null    object 
 2   state             837 non-null    int64  
 3   county            837 non-null    int64  
 4   POP               837 non-null    int64  
 5   EDU_TOT           826 non-null    float64
 6   LESS_HS_TOT       826 non-null    float64
 7   HS_TOT            826 non-null    float64
 8   COL_OR_ASSOC_TOT  826 non-null    float64
 9   BACH_TOT          826 non-null    float64
 10  GRAD_TOT          826 non-null    float64
 11  VEHICLE_TOT       837 non-null    int64  
 12  WALK_TOT          226 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.1+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [84]:
# educ_vehicle_2017.drop('Unnamed: 0', axis=1, inplace=True)
educ_vehicle_2017['WALK_TOT'].unique()
educ_vehicle_2017['WALK_TOT'].isnull().sum()

#drop walk b/c too many missing values and only retain counties and drop also drop that row index column
educ_vehicle_2017.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2017 = remove_col(educ_vehicle_2017, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [85]:
edu_vehicle_estimates = educ_vehicle_2017.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2017[est + '_ratio'] = educ_vehicle_2017[est] / educ_vehicle_2017['POP'] * 100
    # educ_vehicle_2017.drop(est, axis=1, inplace=True)

# convert into ratios
display(educ_vehicle_2017)

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"St. Lawrence County, New York",36,89,109623,72555.0,9164.0,25858.0,20091.0,8279.0,9163.0,35920,66.185928,8.359560,23.588116,18.327358,7.552247,8.358647,32.766846
1,"Saratoga County, New York",36,91,229869,164414.0,10656.0,39033.0,45561.0,37886.0,31278.0,105705,71.525086,4.635684,16.980541,19.820419,16.481561,13.606880,45.984887
2,"Schenectady County, New York",36,93,155565,107453.0,10085.0,32144.0,35263.0,17210.0,12751.0,60290,69.072735,6.482821,20.662745,22.667695,11.062900,8.196574,38.755504
3,"Steuben County, New York",36,101,96281,68011.0,6510.0,25920.0,20685.0,7451.0,7445.0,35585,70.638028,6.761459,26.921199,21.483990,7.738806,7.732574,36.959525
4,"Suffolk County, New York",36,103,1492953,1036946.0,102621.0,277080.0,279213.0,208064.0,169968.0,617715,69.456038,6.873693,18.559191,18.702062,13.936407,11.384685,41.375382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,"Putnam County, New York",36,79,99323,71447.0,5090.0,20597.0,18095.0,14929.0,12736.0,42395,71.933993,5.124694,20.737392,18.218338,15.030758,12.822810,42.683970
833,"Queens County, New York",36,81,2358582,1695302.0,304984.0,493794.0,372885.0,335990.0,187649.0,391980,71.878018,12.930820,20.936054,15.809711,14.245424,7.956009,16.619308
834,"Rensselaer County, New York",36,83,159722,111047.0,9095.0,31730.0,34954.0,20143.0,15125.0,65720,69.525175,5.694269,19.865767,21.884274,12.611287,9.469578,41.146492
835,"Richmond County, New York",36,85,479458,334079.0,39031.0,100730.0,83679.0,66108.0,44531.0,127985,69.678470,8.140650,21.009139,17.452832,13.788069,9.287779,26.693683


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population


# Repeat Steps for education and vehicle 2018 data

In [86]:
educ_vehicle_2018.shape
educ_vehicle_2018.columns

educ_vehicle_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        838 non-null    int64  
 1   NAME              838 non-null    object 
 2   state             838 non-null    int64  
 3   county            838 non-null    int64  
 4   POP               838 non-null    int64  
 5   EDU_TOT           826 non-null    float64
 6   LESS_HS_TOT       826 non-null    float64
 7   HS_TOT            826 non-null    float64
 8   COL_OR_ASSOC_TOT  826 non-null    float64
 9   BACH_TOT          826 non-null    float64
 10  GRAD_TOT          826 non-null    float64
 11  VEHICLE_TOT       838 non-null    int64  
 12  WALK_TOT          231 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.2+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [87]:
educ_vehicle_2018['WALK_TOT'].unique()
educ_vehicle_2018['WALK_TOT'].isnull().sum()

educ_vehicle_2018.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2018 = remove_col(educ_vehicle_2018, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [88]:
educ_vehicle_2018.NAME.duplicated().sum()
educ_vehicle_2018.isnull().sum()
educ_vehicle_2018.loc[educ_vehicle_2018.isnull().any(axis=1)]

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT
376,"Jones County, Mississippi",28,67,68461,NaN,NaN,NaN,NaN,NaN,NaN,26755
827,"Arecibo Municipio, Puerto Rico",72,13,82114,NaN,NaN,NaN,NaN,NaN,NaN,21210
828,"Bayamón Municipio, Puerto Rico",72,21,170480,NaN,NaN,NaN,NaN,NaN,NaN,48105
829,"Caguas Municipio, Puerto Rico",72,25,124434,NaN,NaN,NaN,NaN,NaN,NaN,37810
830,"Carolina Municipio, Puerto Rico",72,31,147661,NaN,NaN,NaN,NaN,NaN,NaN,47205
831,"Guaynabo Municipio, Puerto Rico",72,61,83787,NaN,NaN,NaN,NaN,NaN,NaN,27890
832,"Mayagüez Municipio, Puerto Rico",72,97,71986,NaN,NaN,NaN,NaN,NaN,NaN,14200
833,"Ponce Municipio, Puerto Rico",72,113,133191,NaN,NaN,NaN,NaN,NaN,NaN,32475
834,"San Juan Municipio, Puerto Rico",72,127,320967,NaN,NaN,NaN,NaN,NaN,NaN,94245
835,"Toa Alta Municipio, Puerto Rico",72,135,71094,NaN,NaN,NaN,NaN,NaN,NaN,24755


* All Missing values are Puerto Rico counties similar to previous data sets
* Census Bureau keeps Puerto Rico as separate data column for education
* No duplicated data shown

In [89]:
edu_vehicle_estimates = educ_vehicle_2018.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2018[est + '_ratio'] = educ_vehicle_2018[est] / educ_vehicle_2018['POP'] * 100

# convert into ratios
display(educ_vehicle_2018)

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"Baldwin County, Alabama",1,3,218022,155691.0,15282.0,44013.0,47573.0,32210.0,16613.0,87745,71.410683,7.009384,20.187412,21.820275,14.773738,7.619873,40.245938
1,"Calhoun County, Alabama",1,15,114277,79172.0,12121.0,25761.0,27039.0,7914.0,6337.0,42760,69.280783,10.606684,22.542594,23.660929,6.925278,5.545298,37.417853
2,"Cullman County, Alabama",1,43,83442,58319.0,10228.0,18149.0,22883.0,4916.0,2143.0,33065,69.891661,12.257616,21.750437,27.423839,5.891517,2.568251,39.626327
3,"DeKalb County, Alabama",1,49,71385,48169.0,11681.0,14333.0,14995.0,3931.0,3229.0,26150,67.477761,16.363382,20.078448,21.005814,5.506759,4.523359,36.632346
4,"Elmore County, Alabama",1,51,81887,56793.0,7490.0,18457.0,16505.0,10146.0,4195.0,31600,69.355331,9.146751,22.539597,20.155824,12.390245,5.122913,38.589764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,"Ponce Municipio, Puerto Rico",72,113,133191,NaN,NaN,NaN,NaN,NaN,NaN,32475,NaN,NaN,NaN,NaN,NaN,NaN,24.382278
834,"San Juan Municipio, Puerto Rico",72,127,320967,NaN,NaN,NaN,NaN,NaN,NaN,94245,NaN,NaN,NaN,NaN,NaN,NaN,29.362832
835,"Toa Alta Municipio, Puerto Rico",72,135,71094,NaN,NaN,NaN,NaN,NaN,NaN,24755,NaN,NaN,NaN,NaN,NaN,NaN,34.820097
836,"Toa Baja Municipio, Puerto Rico",72,137,74623,NaN,NaN,NaN,NaN,NaN,NaN,23700,NaN,NaN,NaN,NaN,NaN,NaN,31.759645


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population


# Repeat Steps for education and vehicle 2019 data

In [90]:
educ_vehicle_2019.shape
educ_vehicle_2019.columns

educ_vehicle_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        840 non-null    int64  
 1   NAME              840 non-null    object 
 2   state             840 non-null    int64  
 3   county            840 non-null    int64  
 4   POP               840 non-null    int64  
 5   EDU_TOT           827 non-null    float64
 6   LESS_HS_TOT       827 non-null    float64
 7   HS_TOT            827 non-null    float64
 8   COL_OR_ASSOC_TOT  827 non-null    float64
 9   BACH_TOT          827 non-null    float64
 10  GRAD_TOT          827 non-null    float64
 11  VEHICLE_TOT       840 non-null    int64  
 12  WALK_TOT          226 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.4+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [91]:
educ_vehicle_2019['WALK_TOT'].unique()
educ_vehicle_2019['WALK_TOT'].isnull().sum()

educ_vehicle_2019.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2019 = remove_col(educ_vehicle_2019, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [92]:
educ_vehicle_2019.NAME.duplicated().sum()
educ_vehicle_2019.isnull().sum()
educ_vehicle_2019.loc[educ_vehicle_2019.isnull().any(axis=1)]

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT
291,"St. Landry Parish, Louisiana",22,97,82124,NaN,NaN,NaN,NaN,NaN,NaN,27175
559,"Muskingum County, Ohio",39,119,86215,NaN,NaN,NaN,NaN,NaN,NaN,33710
829,"Arecibo Municipio, Puerto Rico",72,13,81966,NaN,NaN,NaN,NaN,NaN,NaN,20780
830,"Bayamón Municipio, Puerto Rico",72,21,169269,NaN,NaN,NaN,NaN,NaN,NaN,53730
831,"Caguas Municipio, Puerto Rico",72,25,124606,NaN,NaN,NaN,NaN,NaN,NaN,39305
832,"Carolina Municipio, Puerto Rico",72,31,146984,NaN,NaN,NaN,NaN,NaN,NaN,48385
833,"Guaynabo Municipio, Puerto Rico",72,61,83728,NaN,NaN,NaN,NaN,NaN,NaN,30465
834,"Mayagüez Municipio, Puerto Rico",72,97,71530,NaN,NaN,NaN,NaN,NaN,NaN,16695
835,"Ponce Municipio, Puerto Rico",72,113,131881,NaN,NaN,NaN,NaN,NaN,NaN,31635
836,"San Juan Municipio, Puerto Rico",72,127,318441,NaN,NaN,NaN,NaN,NaN,NaN,92055


* All Missing values are Puerto Rico counties similar to previous data sets
* Census Bureau keeps Puerto Rico as separate data column for education
* No duplicated data shown

In [93]:
edu_vehicle_estimates = educ_vehicle_2019.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2019[est + '_ratio'] = educ_vehicle_2019[est] / educ_vehicle_2019['POP'] * 100

# convert into ratios
display(educ_vehicle_2019)

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"Baldwin County, Alabama",1,3,223234,159717.0,15158.0,41884.0,51204.0,32781.0,18690.0,85170,71.546897,6.790184,18.762375,22.937366,14.684591,8.372381,38.152790
1,"Calhoun County, Alabama",1,15,113605,79084.0,12195.0,27277.0,24355.0,9082.0,6175.0,41330,69.613133,10.734563,24.010387,21.438317,7.994366,5.435500,36.380441
2,"Cullman County, Alabama",1,43,83768,58795.0,11014.0,17924.0,20616.0,5935.0,3306.0,31840,70.187900,13.148219,21.397192,24.610830,7.085044,3.946614,38.009741
3,"DeKalb County, Alabama",1,49,71513,47007.0,10043.0,17881.0,13084.0,3693.0,2306.0,28065,65.732105,14.043600,25.003845,18.295974,5.164096,3.224589,39.244613
4,"Elmore County, Alabama",1,51,81209,57553.0,5940.0,19992.0,17311.0,9418.0,4892.0,33005,70.870224,7.314460,24.617961,21.316603,11.597237,6.023963,40.642047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,"Ponce Municipio, Puerto Rico",72,113,131881,NaN,NaN,NaN,NaN,NaN,NaN,31635,NaN,NaN,NaN,NaN,NaN,NaN,23.987534
836,"San Juan Municipio, Puerto Rico",72,127,318441,NaN,NaN,NaN,NaN,NaN,NaN,92055,NaN,NaN,NaN,NaN,NaN,NaN,28.908024
837,"Toa Alta Municipio, Puerto Rico",72,135,72025,NaN,NaN,NaN,NaN,NaN,NaN,24230,NaN,NaN,NaN,NaN,NaN,NaN,33.641097
838,"Toa Baja Municipio, Puerto Rico",72,137,74271,NaN,NaN,NaN,NaN,NaN,NaN,22900,NaN,NaN,NaN,NaN,NaN,NaN,30.833030


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population

## Retrieve NE demographic data

In [94]:
NE_counties = get_NE_States(county_demographic, 'STATE')
NE_dems = county_demographic.loc[NE_counties]

NE_dems = NE_dems.loc[NE_dems.YEAR > 9]
# keep only totals of dem; disregard ages
NE_dems = NE_dems.loc[NE_dems.AGEGRP == 0]
NE_dems

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
70395,50,9,1,Connecticut,Fairfield County,10,0,943038,459242,483796,...,82722,80901,9880,10463,2811,2488,1073,888,724,527
70414,50,9,1,Connecticut,Fairfield County,11,0,943971,459794,484177,...,84420,82945,10250,10816,2898,2645,1119,934,712,562
70433,50,9,1,Connecticut,Fairfield County,12,0,943332,459410,483922,...,85628,84584,10644,11169,3010,2699,1130,940,728,563
70623,50,9,3,Connecticut,Hartford County,10,0,893076,433833,459243,...,65718,68349,12455,13855,2621,2676,1148,1083,611,452
70642,50,9,3,Connecticut,Hartford County,11,0,892580,433437,459143,...,67247,70124,12699,14250,2688,2792,1182,1084,632,463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642694,50,50,25,Vermont,Windham County,11,0,42635,20888,21747,...,499,430,51,63,34,29,11,10,8,2
642713,50,50,25,Vermont,Windham County,12,0,42222,20663,21559,...,491,433,50,62,31,28,14,12,5,2
642903,50,50,27,Vermont,Windsor County,10,0,55224,26961,28263,...,420,418,39,37,28,32,15,14,3,3
642922,50,50,27,Vermont,Windsor County,11,0,55204,26986,28218,...,467,431,42,34,30,33,13,10,5,2


* NE_dems contains all county demographic data for North Eastern US
* Only 2017-2019 data is retained
* All age groups are considered for ethnicities

In [95]:
demographic = NE_dems.columns.values[8:]
NE_dem_ratio = NE_dems

for dem in demographic:
    NE_dem_ratio[dem + '_ratio'] = NE_dems[dem] / NE_dems['TOT_POP'] * 100
    # NE_dem_ratio.rename(columns={dem:dem + '_ratio'}, inplace=True)

display(NE_dem_ratio)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE_ratio,HWAC_FEMALE_ratio,HBAC_MALE_ratio,HBAC_FEMALE_ratio,HIAC_MALE_ratio,HIAC_FEMALE_ratio,HAAC_MALE_ratio,HAAC_FEMALE_ratio,HNAC_MALE_ratio,HNAC_FEMALE_ratio
70395,50,9,1,Connecticut,Fairfield County,10,0,943038,459242,483796,...,8.771863,8.578764,1.047678,1.109499,0.298079,0.263828,0.113781,0.094164,0.076773,0.055883
70414,50,9,1,Connecticut,Fairfield County,11,0,943971,459794,484177,...,8.943071,8.786817,1.085838,1.145798,0.307001,0.280199,0.118542,0.098944,0.075426,0.059536
70433,50,9,1,Connecticut,Fairfield County,12,0,943332,459410,483922,...,9.077186,8.966514,1.128341,1.183995,0.319082,0.286113,0.119788,0.099647,0.077173,0.059682
70623,50,9,3,Connecticut,Hartford County,10,0,893076,433833,459243,...,7.358612,7.653212,1.394618,1.551380,0.293480,0.299639,0.128544,0.121266,0.068415,0.050612
70642,50,9,3,Connecticut,Hartford County,11,0,892580,433437,459143,...,7.534003,7.856327,1.422730,1.596496,0.301149,0.312801,0.132425,0.121446,0.070806,0.051872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642694,50,50,25,Vermont,Windham County,11,0,42635,20888,21747,...,1.170400,1.008561,0.119620,0.147766,0.079747,0.068019,0.025800,0.023455,0.018764,0.004691
642713,50,50,25,Vermont,Windham County,12,0,42222,20663,21559,...,1.162901,1.025532,0.118422,0.146843,0.073421,0.066316,0.033158,0.028421,0.011842,0.004737
642903,50,50,27,Vermont,Windsor County,10,0,55224,26961,28263,...,0.760539,0.756917,0.070621,0.067000,0.050703,0.057946,0.027162,0.025351,0.005432,0.005432
642922,50,50,27,Vermont,Windsor County,11,0,55204,26986,28218,...,0.845953,0.780741,0.076081,0.061590,0.054344,0.059778,0.023549,0.018115,0.009057,0.003623


* Columns of demographic ratios are produced
* Ratios are demographic / county population


In [96]:
NE_dem_ratio = get_df_with_geofips(NE_dem_ratio, 'STATE', 'COUNTY')

In [97]:
cols_to_remove =  ['AGEGRP', 'STATE', 'COUNTY', 'SUMLEV']
NE_dem_ratio = remove_col(NE_dem_ratio, cols_to_remove)
display(NE_dem_ratio)

,GeoFIPS,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,...,HWAC_MALE_ratio,HWAC_FEMALE_ratio,HBAC_MALE_ratio,HBAC_FEMALE_ratio,HIAC_MALE_ratio,HIAC_FEMALE_ratio,HAAC_MALE_ratio,HAAC_FEMALE_ratio,HNAC_MALE_ratio,HNAC_FEMALE_ratio
70395,09001,Connecticut,Fairfield County,10,943038,459242,483796,366178,379174,54885,...,8.771863,8.578764,1.047678,1.109499,0.298079,0.263828,0.113781,0.094164,0.076773,0.055883
70414,09001,Connecticut,Fairfield County,11,943971,459794,484177,365198,378062,55522,...,8.943071,8.786817,1.085838,1.145798,0.307001,0.280199,0.118542,0.098944,0.075426,0.059536
70433,09001,Connecticut,Fairfield County,12,943332,459410,483922,363173,376006,56464,...,9.077186,8.966514,1.128341,1.183995,0.319082,0.286113,0.119788,0.099647,0.077173,0.059682
70623,09003,Connecticut,Hartford County,10,893076,433833,459243,327876,347484,66613,...,7.358612,7.653212,1.394618,1.551380,0.293480,0.299639,0.128544,0.121266,0.068415,0.050612
70642,09003,Connecticut,Hartford County,11,892580,433437,459143,325963,345490,66792,...,7.534003,7.856327,1.422730,1.596496,0.301149,0.312801,0.132425,0.121446,0.070806,0.051872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642694,50025,Vermont,Windham County,11,42635,20888,21747,19801,20614,377,...,1.170400,1.008561,0.119620,0.147766,0.079747,0.068019,0.025800,0.023455,0.018764,0.004691
642713,50025,Vermont,Windham County,12,42222,20663,21559,19564,20421,395,...,1.162901,1.025532,0.118422,0.146843,0.073421,0.066316,0.033158,0.028421,0.011842,0.004737
642903,50027,Vermont,Windsor County,10,55224,26961,28263,25896,27192,279,...,0.760539,0.756917,0.070621,0.067000,0.050703,0.057946,0.027162,0.025351,0.005432,0.005432
642922,50027,Vermont,Windsor County,11,55204,26986,28218,25881,27149,286,...,0.845953,0.780741,0.076081,0.061590,0.054344,0.059778,0.023549,0.018115,0.009057,0.003623


* GeoFIPS column added
* Redundant column removes and columns with lack of data variety

In [98]:
NE_dem_ratio_17 = NE_dem_ratio.loc[NE_dems.YEAR == 10]
NE_dem_ratio_18 = NE_dem_ratio.loc[NE_dems.YEAR == 11]
NE_dem_ratio_19 = NE_dem_ratio.loc[NE_dems.YEAR == 12]

NE_dem_ratio_17.drop('YEAR', axis=1, inplace=True)
NE_dem_ratio_18.drop('YEAR', axis=1, inplace=True)
NE_dem_ratio_19.drop('YEAR', axis=1, inplace=True)


/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Merge data

## merge poverty rates for 2017-2019 into one dataframe

In [99]:
pov_cols

for col in pov_cols:
    pov_rate_2017.rename(columns={col:col + '_2017'}, inplace=True)
    pov_rate_2018.rename(columns={col:col + '_2018'}, inplace=True)
    pov_rate_2019.rename(columns={col:col + '_2019'}, inplace=True)

# pov_rate_2017.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2017'}, inplace=True)
# pov_rate_2018.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2018'}, inplace=True)
# pov_rate_2019.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2019'}, inplace=True)
#
pov_rate_2017_2018 = pd.merge(pov_rate_2017, pov_rate_2018, how='inner')
pov_rate_2017_2019 = pd.merge(pov_rate_2017_2018, pov_rate_2019, how='inner')
pov_rate_2017_2019

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages_2017",90% CI LB All Ages_2017,90% CI UB All Ages_2017,"Poverty Percent, All Ages_2017",90% CI LB percent_2017,90% CI UB percent_2017,...,90% CI UB 0-17 percent_2019,"Poverty Estimate, Age 5-17 in Families_2019",90% CI LB 5-17 fam_2019,90% CI UB 5-17 fam_2019,"Poverty Percent, Age 5-17 in Families_2019",90% CI LB 5-17 percent_2019,90% CI UB percent 5-17 percent_2019,Median Household Income_2019,90% CI Lower Bound LB medh inc_2019,90% CI UB medh inc_2019
0,01,001,AL,Autauga County,7390.0,6147.0,8633.0,13.400000,11.1,15.700000,...,20.299999,1376.0,902.0,1850.0,14.400000,9.400000,19.400000,58233.0,52517.0,63949.0
1,01,003,AL,Baldwin County,21199.0,17444.0,24954.0,10.100000,8.3,11.900000,...,17.400000,4641.0,3295.0,5987.0,13.300000,9.400000,17.200001,59871.0,54593.0,65149.0
2,01,005,AL,Barbour County,7414.0,6325.0,8503.0,33.400002,28.5,38.299999,...,50.799999,1468.0,1114.0,1822.0,39.500000,30.000000,49.000000,35972.0,31822.0,40122.0
3,01,007,AL,Bibb County,4137.0,3187.0,5087.0,20.200001,15.5,24.900000,...,33.099998,808.0,566.0,1050.0,25.200001,17.700001,32.700001,47918.0,42291.0,53545.0
4,01,009,AL,Blount County,7343.0,5805.0,8881.0,12.800000,10.1,15.500000,...,25.299999,2067.0,1641.0,2493.0,21.400000,17.000000,25.799999,52902.0,46777.0,59027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,56,037,WY,Sweetwater County,4253.0,3540.0,4966.0,10.000000,8.3,11.700000,...,13.000000,644.0,416.0,872.0,8.200000,5.300000,11.100000,80639.0,73437.0,87841.0
3002,56,039,WY,Teton County,1348.0,1021.0,1675.0,5.800000,4.4,7.200000,...,7.800000,142.0,83.0,201.0,4.700000,2.700000,6.700000,98837.0,86531.0,111143.0
3003,56,041,WY,Uinta County,2067.0,1623.0,2511.0,10.200000,8.0,12.400000,...,13.100000,336.0,193.0,479.0,7.800000,4.500000,11.100000,70756.0,63191.0,78321.0
3004,56,043,WY,Washakie County,845.0,632.0,1058.0,10.700000,8.0,13.400000,...,18.900000,160.0,94.0,226.0,12.300000,7.200000,17.400000,55122.0,50050.0,60194.0


In [100]:
state_code = pov_rate_2017_2019['State FIPS Code'].astype(str).values
county_code = pov_rate_2017_2019['County FIPS Code'].astype(str).values
geofips = []
for stateID, countyID in zip(state_code, county_code):
    id = ''
    id += stateID
    id += countyID
    geofips.append(id)
pov_rate_2017_2019.insert(loc=0, column='GeoFIPS', value=geofips)

In [101]:
# pov_rate_2017_2019.drop(['State FIPS Code', 'County FIPS Code'], axis=1, inplace=True)
NE_States = (pov_rate_2017_2019['Postal Code'] == 'MA') | \
            (pov_rate_2017_2019['Postal Code'] == 'CT') | \
            (pov_rate_2017_2019['Postal Code'] == 'ME') | \
            (pov_rate_2017_2019['Postal Code'] == 'NY') | \
            (pov_rate_2017_2019['Postal Code'] == 'NH') | \
            (pov_rate_2017_2019['Postal Code'] == 'NJ') | \
            (pov_rate_2017_2019['Postal Code'] == 'PA') | \
            (pov_rate_2017_2019['Postal Code'] == 'VT') | \
            (pov_rate_2017_2019['Postal Code'] == 'DE') | \
            (pov_rate_2017_2019['Postal Code'] == 'MD') | \
            (pov_rate_2017_2019['Postal Code'] == 'RI') | \
            (pov_rate_2017_2019['Postal Code'] == 'DC')

NE_pov_rate_2017_2019 = pov_rate_2017_2019.loc[NE_States]
NE_pov_rate_2017_2019.drop(['State FIPS Code', 'County FIPS Code'], axis=1, inplace=True)
display(NE_pov_rate_2017_2019)

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,GeoFIPS,Postal Code,Name,"Poverty Estimate, All Ages_2017",90% CI LB All Ages_2017,90% CI UB All Ages_2017,"Poverty Percent, All Ages_2017",90% CI LB percent_2017,90% CI UB percent_2017,"Poverty Estimate, Age 0-17_2017",...,90% CI UB 0-17 percent_2019,"Poverty Estimate, Age 5-17 in Families_2019",90% CI LB 5-17 fam_2019,90% CI UB 5-17 fam_2019,"Poverty Percent, Age 5-17 in Families_2019",90% CI LB 5-17 percent_2019,90% CI UB percent 5-17 percent_2019,Median Household Income_2019,90% CI Lower Bound LB medh inc_2019,90% CI UB medh inc_2019
279,09001,CT,Fairfield County,82428.0,74608.0,90248.0,8.8,8.0,9.600000,24161.0,...,13.300000,18562.0,16078.0,21046.0,11.900000,10.3,13.500000,96966.0,93161.0,100771.0
280,09003,CT,Hartford County,96200.0,88256.0,104144.0,11.0,10.1,11.900000,28488.0,...,16.000000,17852.0,15166.0,20538.0,13.100000,11.1,15.100000,75336.0,73281.0,77391.0
281,09005,CT,Litchfield County,12481.0,10381.0,14581.0,6.9,5.7,8.100000,2642.0,...,11.600000,2077.0,1591.0,2563.0,8.500000,6.5,10.500000,81015.0,77226.0,84804.0
282,09007,CT,Middlesex County,10796.0,8906.0,12686.0,6.8,5.6,8.000000,2169.0,...,9.100000,1388.0,981.0,1795.0,6.600000,4.7,8.500000,81721.0,74613.0,88829.0
283,09009,CT,New Haven County,91639.0,82523.0,100755.0,11.0,9.9,12.100000,26499.0,...,20.500000,22220.0,19581.0,24859.0,17.799999,15.7,19.900000,69687.0,66999.0,72375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717,50019,VT,Orleans County,3962.0,3189.0,4735.0,15.2,12.2,18.200001,1007.0,...,23.299999,643.0,427.0,859.0,16.700001,11.1,22.299999,48826.0,43855.0,53797.0
2718,50021,VT,Rutland County,6106.0,4691.0,7521.0,10.8,8.3,13.300000,1518.0,...,16.400000,849.0,480.0,1218.0,11.400000,6.5,16.299999,51903.0,49484.0,54322.0
2719,50023,VT,Washington County,5252.0,4184.0,6320.0,9.4,7.5,11.300000,1114.0,...,12.700000,636.0,373.0,899.0,8.000000,4.7,11.300000,65879.0,61501.0,70257.0
2720,50025,VT,Windham County,4827.0,3819.0,5835.0,11.6,9.2,14.000000,1138.0,...,17.900000,664.0,410.0,918.0,12.500000,7.7,17.299999,52068.0,47230.0,56906.0


## merge education and vehicle rates for 2017-2019 into one dataframe

In [102]:
educ_vehicle_ratios = educ_vehicle_2017.columns[3:]
educ_vehicle_ratios
for ratio in educ_vehicle_ratios:
    educ_vehicle_2017.rename(columns={ratio:ratio + '_2017'}, inplace=True)
    educ_vehicle_2018.rename(columns={ratio:ratio + '_2018'}, inplace=True)
    educ_vehicle_2019.rename(columns={ratio:ratio + '_2019'}, inplace=True)

# educ_vehicle_2017.drop('POP', axis=1, inplace=True)
# educ_vehicle_2018.drop('POP', axis=1, inplace=True)
# educ_vehicle_2019.drop('POP', axis=1, inplace=True)

In [103]:
educ_vehicle_2017_2018 = pd.merge(educ_vehicle_2017, educ_vehicle_2018, how='inner')
educ_vehicle_2017_2019 = pd.merge(educ_vehicle_2017_2018, educ_vehicle_2019, how='inner')
educ_vehicle_2017_2019

,NAME,state,county,POP_2017,EDU_TOT_2017,LESS_HS_TOT_2017,HS_TOT_2017,COL_OR_ASSOC_TOT_2017,BACH_TOT_2017,GRAD_TOT_2017,...,BACH_TOT_2019,GRAD_TOT_2019,VEHICLE_TOT_2019,EDU_TOT_ratio_2019,LESS_HS_TOT_ratio_2019,HS_TOT_ratio_2019,COL_OR_ASSOC_TOT_ratio_2019,BACH_TOT_ratio_2019,GRAD_TOT_ratio_2019,VEHICLE_TOT_ratio_2019
0,"St. Lawrence County, New York",36,89,109623,72555.0,9164.0,25858.0,20091.0,8279.0,9163.0,...,8684.0,7733.0,33650,67.044737,8.564136,23.762762,19.480230,8.060145,7.177464,31.232597
1,"Saratoga County, New York",36,91,229869,164414.0,10656.0,39033.0,45561.0,37886.0,31278.0,...,40148.0,30363.0,104085,72.095118,4.492676,17.754924,19.172290,17.466056,13.209172,45.281320
2,"Schenectady County, New York",36,93,155565,107453.0,10085.0,32144.0,35263.0,17210.0,12751.0,...,22627.0,15566.0,59970,70.183324,5.047682,20.198456,20.343982,14.569959,10.023245,38.615831
3,"Steuben County, New York",36,101,96281,68011.0,6510.0,25920.0,20685.0,7451.0,7445.0,...,8066.0,8475.0,35505,71.044989,6.093585,25.791841,21.817171,8.456788,8.885604,37.225175
4,"Suffolk County, New York",36,103,1492953,1036946.0,102621.0,277080.0,279213.0,208064.0,169968.0,...,214035.0,182346.0,629360,70.148605,6.397598,18.389192,18.517663,14.495114,12.349037,42.622211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,"Putnam County, New York",36,79,99323,71447.0,5090.0,20597.0,18095.0,14929.0,12736.0,...,15240.0,12261.0,42420,73.443857,5.721115,19.355167,20.396664,15.500407,12.470504,43.144833
832,"Queens County, New York",36,81,2358582,1695302.0,304984.0,493794.0,372885.0,335990.0,187649.0,...,334272.0,207624.0,387870,72.484558,12.328372,19.812340,16.300805,14.831103,9.211938,17.209159
833,"Rensselaer County, New York",36,83,159722,111047.0,9095.0,31730.0,34954.0,20143.0,15125.0,...,20896.0,18203.0,65615,70.327759,5.902441,19.919478,19.870963,13.165820,11.469058,41.341659
834,"Richmond County, New York",36,85,479458,334079.0,39031.0,100730.0,83679.0,66108.0,44531.0,...,71914.0,48875.0,128740,70.041143,7.701258,20.426427,16.545240,15.103446,10.264773,27.038096


In [104]:
educ_vehicle_2017_2019 = get_df_with_geofips(educ_vehicle_2017_2019, 'state', 'county')
educ_vehicle_2017_2019

,GeoFIPS,NAME,state,county,POP_2017,EDU_TOT_2017,LESS_HS_TOT_2017,HS_TOT_2017,COL_OR_ASSOC_TOT_2017,BACH_TOT_2017,...,BACH_TOT_2019,GRAD_TOT_2019,VEHICLE_TOT_2019,EDU_TOT_ratio_2019,LESS_HS_TOT_ratio_2019,HS_TOT_ratio_2019,COL_OR_ASSOC_TOT_ratio_2019,BACH_TOT_ratio_2019,GRAD_TOT_ratio_2019,VEHICLE_TOT_ratio_2019
0,36089,"St. Lawrence County, New York",36,89,109623,72555.0,9164.0,25858.0,20091.0,8279.0,...,8684.0,7733.0,33650,67.044737,8.564136,23.762762,19.480230,8.060145,7.177464,31.232597
1,36091,"Saratoga County, New York",36,91,229869,164414.0,10656.0,39033.0,45561.0,37886.0,...,40148.0,30363.0,104085,72.095118,4.492676,17.754924,19.172290,17.466056,13.209172,45.281320
2,36093,"Schenectady County, New York",36,93,155565,107453.0,10085.0,32144.0,35263.0,17210.0,...,22627.0,15566.0,59970,70.183324,5.047682,20.198456,20.343982,14.569959,10.023245,38.615831
3,36101,"Steuben County, New York",36,101,96281,68011.0,6510.0,25920.0,20685.0,7451.0,...,8066.0,8475.0,35505,71.044989,6.093585,25.791841,21.817171,8.456788,8.885604,37.225175
4,36103,"Suffolk County, New York",36,103,1492953,1036946.0,102621.0,277080.0,279213.0,208064.0,...,214035.0,182346.0,629360,70.148605,6.397598,18.389192,18.517663,14.495114,12.349037,42.622211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,36079,"Putnam County, New York",36,79,99323,71447.0,5090.0,20597.0,18095.0,14929.0,...,15240.0,12261.0,42420,73.443857,5.721115,19.355167,20.396664,15.500407,12.470504,43.144833
832,36081,"Queens County, New York",36,81,2358582,1695302.0,304984.0,493794.0,372885.0,335990.0,...,334272.0,207624.0,387870,72.484558,12.328372,19.812340,16.300805,14.831103,9.211938,17.209159
833,36083,"Rensselaer County, New York",36,83,159722,111047.0,9095.0,31730.0,34954.0,20143.0,...,20896.0,18203.0,65615,70.327759,5.902441,19.919478,19.870963,13.165820,11.469058,41.341659
834,36085,"Richmond County, New York",36,85,479458,334079.0,39031.0,100730.0,83679.0,66108.0,...,71914.0,48875.0,128740,70.041143,7.701258,20.426427,16.545240,15.103446,10.264773,27.038096


In [105]:
NE_states = get_NE_States(educ_vehicle_2017_2019, 'state')
NE_educ_vehicle_2017_2019 = educ_vehicle_2017_2019.loc[NE_states]
display(NE_educ_vehicle_2017_2019)

,GeoFIPS,NAME,state,county,POP_2017,EDU_TOT_2017,LESS_HS_TOT_2017,HS_TOT_2017,COL_OR_ASSOC_TOT_2017,BACH_TOT_2017,...,BACH_TOT_2019,GRAD_TOT_2019,VEHICLE_TOT_2019,EDU_TOT_ratio_2019,LESS_HS_TOT_ratio_2019,HS_TOT_ratio_2019,COL_OR_ASSOC_TOT_ratio_2019,BACH_TOT_ratio_2019,GRAD_TOT_ratio_2019,VEHICLE_TOT_ratio_2019
0,36089,"St. Lawrence County, New York",36,89,109623,72555.0,9164.0,25858.0,20091.0,8279.0,...,8684.0,7733.0,33650,67.044737,8.564136,23.762762,19.480230,8.060145,7.177464,31.232597
1,36091,"Saratoga County, New York",36,91,229869,164414.0,10656.0,39033.0,45561.0,37886.0,...,40148.0,30363.0,104085,72.095118,4.492676,17.754924,19.172290,17.466056,13.209172,45.281320
2,36093,"Schenectady County, New York",36,93,155565,107453.0,10085.0,32144.0,35263.0,17210.0,...,22627.0,15566.0,59970,70.183324,5.047682,20.198456,20.343982,14.569959,10.023245,38.615831
3,36101,"Steuben County, New York",36,101,96281,68011.0,6510.0,25920.0,20685.0,7451.0,...,8066.0,8475.0,35505,71.044989,6.093585,25.791841,21.817171,8.456788,8.885604,37.225175
4,36103,"Suffolk County, New York",36,103,1492953,1036946.0,102621.0,277080.0,279213.0,208064.0,...,214035.0,182346.0,629360,70.148605,6.397598,18.389192,18.517663,14.495114,12.349037,42.622211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,36079,"Putnam County, New York",36,79,99323,71447.0,5090.0,20597.0,18095.0,14929.0,...,15240.0,12261.0,42420,73.443857,5.721115,19.355167,20.396664,15.500407,12.470504,43.144833
832,36081,"Queens County, New York",36,81,2358582,1695302.0,304984.0,493794.0,372885.0,335990.0,...,334272.0,207624.0,387870,72.484558,12.328372,19.812340,16.300805,14.831103,9.211938,17.209159
833,36083,"Rensselaer County, New York",36,83,159722,111047.0,9095.0,31730.0,34954.0,20143.0,...,20896.0,18203.0,65615,70.327759,5.902441,19.919478,19.870963,13.165820,11.469058,41.341659
834,36085,"Richmond County, New York",36,85,479458,334079.0,39031.0,100730.0,83679.0,66108.0,...,71914.0,48875.0,128740,70.041143,7.701258,20.426427,16.545240,15.103446,10.264773,27.038096


## Merge demographic years as one spanning 2017-2019


In [106]:
# add suffix to each column name then merge
demographic = NE_dem_ratio_17.columns.values[3:]

for dem in demographic:
    NE_dem_ratio_17.rename(columns={dem:dem + '_2017'}, inplace=True)
    NE_dem_ratio_18.rename(columns={dem:dem + '_2018'}, inplace=True)
    NE_dem_ratio_19.rename(columns={dem:dem + '_2019'}, inplace=True)

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [107]:
NE_dem_ratio_2017_2018 = pd.merge(NE_dem_ratio_17, NE_dem_ratio_18, how='inner')
NE_dem_ratio_2017_2019 = pd.merge(NE_dem_ratio_2017_2018, NE_dem_ratio_19, how='inner')
NE_dem_ratio_2017_2019.shape
display(NE_dem_ratio_2017_2019)

,GeoFIPS,STNAME,CTYNAME,TOT_POP_2017,TOT_MALE_2017,TOT_FEMALE_2017,WA_MALE_2017,WA_FEMALE_2017,BA_MALE_2017,BA_FEMALE_2017,...,HWAC_MALE_ratio_2019,HWAC_FEMALE_ratio_2019,HBAC_MALE_ratio_2019,HBAC_FEMALE_ratio_2019,HIAC_MALE_ratio_2019,HIAC_FEMALE_ratio_2019,HAAC_MALE_ratio_2019,HAAC_FEMALE_ratio_2019,HNAC_MALE_ratio_2019,HNAC_FEMALE_ratio_2019
0,09001,Connecticut,Fairfield County,943038,459242,483796,366178,379174,54885,63641,...,9.077186,8.966514,1.128341,1.183995,0.319082,0.286113,0.119788,0.099647,0.077173,0.059682
1,09003,Connecticut,Hartford County,893076,433833,459243,327876,347484,66613,71919,...,7.626834,7.970103,1.458642,1.633585,0.303234,0.315458,0.137711,0.124815,0.069304,0.052819
2,09005,Connecticut,Litchfield County,181667,89749,91918,83969,86459,2095,1760,...,3.141965,3.005551,0.454714,0.417561,0.167468,0.144732,0.053789,0.051017,0.030499,0.022736
3,09007,Connecticut,Middlesex County,162942,79376,83566,70949,74431,4283,4437,...,2.782019,2.829422,0.419858,0.471570,0.088035,0.112044,0.056022,0.052944,0.029550,0.032013
4,09009,Connecticut,New Haven County,857748,413560,444188,324268,344780,59423,66910,...,8.058080,8.240939,1.302943,1.452343,0.285812,0.297628,0.097104,0.095115,0.048786,0.047733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,50019,Vermont,Orleans County,26811,13459,13352,12970,12884,128,83,...,0.828494,0.702741,0.073973,0.070274,0.059178,0.051781,0.022192,0.018493,0.011096,0.007397
241,50021,Vermont,Rutland County,59006,29118,29888,28147,28858,245,148,...,0.740664,0.704576,0.089361,0.044680,0.054991,0.049836,0.024059,0.015466,0.006874,0.005155
242,50023,Vermont,Washington County,58253,28808,29445,27604,28267,334,218,...,0.919379,0.910819,0.089027,0.051362,0.068483,0.053074,0.018833,0.020545,0.006848,0.000000
243,50025,Vermont,Windham County,42851,20998,21853,19944,20751,355,247,...,1.162901,1.025532,0.118422,0.146843,0.073421,0.066316,0.033158,0.028421,0.011842,0.004737


# saved cleaned up datasets


In [108]:

data_path = r'../../data/processed/alt_clean_NE_edu_vehicle_2017_2019.csv'
NE_educ_vehicle_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/alt_clean_NE_pov_rate_2017_2019.csv'
NE_pov_rate_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/alt_clean_NE_unemploy_rate_2017_2019.csv'
NE_unemploy_rate_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/alt_clean_NE_dem_ratio_2017_2019.csv'
NE_dem_ratio_2017_2019.to_csv(data_path, index=False)